## LBC group management


In [5]:
import pandas as pd
import numpy as np
from collections import namedtuple

requirement = namedtuple('requirement', 'target min max')
sc_group_size = requirement(8,6,9)
sc_singles_count = requirement(2,0,4)
sc_couple_count = requirement(3,3,4)
lb_group_size = requirement(6,5,7)
lb_singles_count = requirement(2,0,3)
lb_couple_count = requirement(3,2,3)

In [8]:
group_history = pd.read_csv("LBC_group_history.csv")
group_history.columns = list(f'{"sc" if i%2 else "lb"}{i//2}'
                             for i in range(len(group_history.columns))
                             )
display(group_history.head(5))


,lb0,sc0,lb1,sc1,lb2,sc2,lb3,sc3,lb4,sc4,...,sc9,lb10,sc10,lb11,sc11,lb12,sc12,lb13,sc13,lb14
0,D,NaN,F,NaN,G,NaN,NaN,B,E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,E,F,C,I,B,F,C,F,A,...,A,NaN,A,NaN,NaN,NaN,C,NaN,D,NaN
2,E,NaN,NaN,NaN,G,NaN,C,NaN,D,NaN,...,B,NaN,A,NaN,C,NaN,C,NaN,D,NaN
3,B,D,NaN,NaN,I,C,G,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E,B,NaN,NaN,A,NaN,E,NaN,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
group_features = pd.DataFrame(columns=['how many groups', 'smallest group size', 'largest group size',])

for column in group_history.columns:
    # print(group_history[column])

    group_features.loc[column] = [
        group_history[column].count(),
        group_history[column].value_counts().min(),
        group_history[column].value_counts().max(),
    ]

display(group_features.head(5))

,how many groups,smallest group size,largest group size
lb0,19,2,5
sc0,23,3,5
lb1,20,1,4
sc1,16,4,4
lb2,26,2,4
